# Task 3

Tools Used: Big Query, SQL
* Checked the Excel files and the column names used special characters, so will need to rename them.
* Converting the Excel files to CSV for Big Query



In [5]:
import pandas as pd

In [6]:
supplier1 = pd.read_excel("/content/supplier_data1.xlsx")
supplier2 = pd.read_excel("/content/supplier_data2.xlsx")
buyer_prefs = pd.read_excel("/content/buyer_preferences.xlsx")

In [7]:
supplier1.columns

Index(['Quality/Choice', 'Grade', 'Finish', 'Thickness (mm)', 'Width (mm)',
       'Description', 'Gross weight (kg)', 'RP02', 'RM', 'Quantity', 'AG',
       'AI'],
      dtype='object')

In [8]:
supplier2.columns

Index(['Material', 'Description', 'Article ID', 'Weight (kg)', 'Quantity',
       'Reserved'],
      dtype='object')

In [9]:
buyer_prefs.columns

Index(['Buyer ID', 'Preferred Grade', 'Preferred Finish',
       'Preferred Thickness (mm)', 'Preferred Width (mm)', 'Max Weight (kg)',
       'Min Quantity'],
      dtype='object')

In [10]:
#renaming columns to remove special characters
supplier1.rename(columns={"Quality/Choice": "Quality_Choice", "Thickness (mm)": "Thickness_mm","Width (mm)": "Width_mm",
                          "Gross weight (kg)": "Gross_weight_kg"}, inplace=True)

supplier2.rename(columns={"Weight (kg)": "Weight_kg", "Article ID": "Article_ID"}, inplace=True)

buyer_prefs.rename(columns={"Buyer ID": "Buyer_ID", "Preferred Grade": "Preferred_Grade", "Preferred Finish": "Preferred_Finish",
                            "Preferred Thickness (mm)": "Preferred_Thickness_mm","Preferred Width (mm)": "Preferred_Width_mm",
                            "Max Weight (kg)": "Max_Weight_kg","Min Quantity": "Min_Quantity"}, inplace=True)

In [11]:
#saving the files as csv for Big Query
supplier1.to_csv("supplier_data1.csv", index=False)
supplier2.to_csv("supplier_data2.csv", index=False)
buyer_prefs.to_csv("buyer_prefs.csv", index=False)

## Merging both tables on Big Query

There are only two columns that are common between both tbls:
  1. Quantity
  2. Weight

I will be keeping the non-matching columns as they might be useful later.

```
create or replace table vanillasteel-452307.vanilla_steel.suppliers_merged as  
select 'supplier_1' as supplier_source, -- to be able to differentiate between both tbls
  null as article_id,  
  grade,  
  finish,  
  thickness_mm,  
  width_mm,  
  gross_weight_kg as weight_kg,  
  quantity  
from vanillasteel-452307.vanilla_steel.supplier_data1

union all  

select 'supplier_2' as supplier_source,  
  article_id,  
  null as grade,  
  null as finish,  
  null as thickness_mm,  
  null as width_mm,  
  weight_kg,  
  quantity  
from vanillasteel-452307.vanilla_steel.supplier_data2

```





## Matching Supplier Materials to Buyer Preferences

To create the recommendation table, I used SQL on Big Query. To increase the number of matches, I have kept the join conditions flexible (as both tables have mostly non-matching columns, i.e, many missing values).

I have kept grade, finish, thickness, and width as optional which can still match buyers even if a supplier doesn’t have particular field(s) available, but made weight and quantity info as compulsory for matching as both tables have it and it also is a very basic criteria for a buyer which needs to match.

```
create or replace table vanillasteel-452307.vanilla_steel.recommendations as  
select bp.buyer_id,  
    s.supplier_source,  
    s.article_id,  
    s.grade,  
    s.finish,  
    s.thickness_mm,  
    s.width_mm,  
    s.weight_kg,  
    s.quantity  
from vanillasteel-452307.vanilla_steel.buyer_prefs bp  
join vanillasteel-452307.vanilla_steel.suppliers_merged s  
    on ( 1=1
    or (bp.preferred_grade = s.grade)
    or (bp.preferred_finish = s.finish)
    or (bp.preferred_thickness_mm = s.thickness_mm)
    or (bp.preferred_width_mm = s.width_mm)
    and (s.weight_kg <= bp.max_weight_kg)
    and (s.quantity >= bp.min_quantity))

```

The Recommendations table can be exported as CSV from Big Query, but as the data is quite limited (approx. 300), I'll manually copy-paste it onto an Excel sheet for easier overview.